In [1]:
import pandas as pd
import numpy as np

In [13]:
# org_data = pd.read_excel("2017-ZHU.xlsx",sheet_name='Sheet1')
# predict_data = pd.read_excel("2017-ZHU.xlsx",sheet_name="Sheet2")
org_data = pd.read_excel("input.xlsx",sheet_name="Sheet1",header=None)
predict_data = pd.read_excel("input.xlsx",sheet_name='Sheet2',header=None)

In [34]:
unpredict_a  = np.array(org_data.loc[0:41,0:41])     # 原始数据矩阵
row_sums = np.array(predict_data.loc[0:41,42])       # 42个列和
column_sums = np.array(predict_data.loc[42,0:41])    # 42个行和


In [45]:

def modified_ras(T_initial, row_totals, col_totals, max_iterations=10000, tolerance=1e-10000):
    """
    使用修正RAS方法调整矩阵T_initial，使其行和列的和分别接近row_totals和col_totals。
    
    参数:
    - T_initial: 初始矩阵。
    - row_totals: 目标行总和。
    - col_totals: 目标列总和。
    - max_iterations: 最大迭代次数。
    - tolerance: 收敛容忍度。
    
    返回:
    - T_adjusted: 调整后的矩阵。
    """
    # 初始化
    rows, cols = T_initial.shape
    R = np.ones(rows)
    S = np.ones(cols)
    T_adjusted = np.copy(T_initial)
    
    for iteration in range(max_iterations):
        # 行调整
        row_sums = T_adjusted.sum(axis=1)
        R = row_totals / row_sums
        for i in range(rows):
            T_adjusted[i, :] *= R[i]
        
        # 列调整
        col_sums = T_adjusted.sum(axis=0)
        S = col_totals / col_sums
        for j in range(cols):
            T_adjusted[:, j] *= S[j]
        
        # 检查收敛
        if np.allclose(row_totals, T_adjusted.sum(axis=1), atol=tolerance) and \
           np.allclose(col_totals, T_adjusted.sum(axis=0), atol=tolerance):
            print(f"Converged in {iteration+1} iterations.")
            return T_adjusted
    
    print("Did not converge.")
    return T_adjusted

T_adu = modified_ras(unpredict_a,row_sums,column_sums)
# 示例使用
# T_initial = np.random.rand(4, 4) * 100  # 初始矩阵
# row_totals = np.array([100, 200, 300, 400])  # 目标行总和
# col_totals = np.array([250, 350, 150, 250])  # 目标列总和

# 调整矩阵
# T_adjusted = modified_ras(T_initial, row_totals, col_totals)

# print("调整后的矩阵:")
# print(T_adu)


def calculate_accuracy(T_adjusted, row_totals, col_totals):
    """
    计算调整后矩阵的精度，包括绝对误差和相对误差。
    
    参数:
    - T_adjusted: 调整后的矩阵。
    - row_totals: 目标行总和。
    - col_totals: 目标列总和。
    
    返回:
    - 绝对误差和相对误差的字典。
    """
    adjusted_row_sums = T_adjusted.sum(axis=1)
    adjusted_col_sums = T_adjusted.sum(axis=0)
    
    # 绝对误差
    absolute_error_rows = np.abs(adjusted_row_sums - row_totals)
    absolute_error_cols = np.abs(adjusted_col_sums - col_totals)
    
    # 相对误差
    relative_error_rows = absolute_error_rows / row_totals
    relative_error_cols = absolute_error_cols / col_totals
    
    accuracy = {
        'Absolute Error (Rows)': absolute_error_rows,
        'Relative Error (Rows)': relative_error_rows,
        'Absolute Error (Cols)': absolute_error_cols,
        'Relative Error (Cols)': relative_error_cols,
    }
    
    return accuracy

# 计算精度
accuracy = calculate_accuracy(T_adu, row_sums, column_sums)

# 输出精度信息
for key, value in accuracy.items():
    print(f"{key}: {value.mean()}")


Converged in 16 iterations.
调整后的矩阵:
[[1.43236753e+08 2.16029344e+05 3.04159666e+03 ... 3.73875221e+05
  4.05618026e+05 0.00000000e+00]
 [5.98457164e+05 3.65882707e+07 8.99624610e+04 ... 4.15172655e+05
  4.26613095e+04 5.77003372e+05]
 [0.00000000e+00 7.00770707e+04 1.04429414e+06 ... 5.89806865e+04
  2.33011867e+03 6.65985038e+04]
 ...
 [1.17704665e+05 1.50850792e+05 1.75223014e+04 ... 2.41611352e+06
  5.94672567e+04 2.50766068e+06]
 [8.63236884e+04 2.51987109e+05 1.53679871e+05 ... 3.07855678e+05
  9.38415104e+06 7.98961913e+06]
 [3.98089654e+05 6.67738041e+04 2.90893425e+04 ... 1.45877374e+05
  1.64119651e+05 1.77718823e+07]]
Absolute Error (Rows): 505.4989506845845
Relative Error (Rows): 1.1931100451130956e-06
Absolute Error (Cols): 4.204256193978446e-08
Relative Error (Cols): 1.0550123730792673e-16


In [43]:
T_adu_df = pd.DataFrame(T_adu)
T_adu_df.to_excel('output.xlsx',index=False)